In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import requests

from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
url = 'https://haslametrics.com/'
options = Options()
options.headless = True
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Could not get version for google-chrome with the command:  powershell "$ErrorActionPreference='silentlycontinue' ; (Get-Item -Path "$env:PROGRAMFILES\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion ; if (-not $? -or $? -match $error) { (Get-Item -Path "$env:PROGRAMFILES(x86)\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { (Get-Item -Path "$env:LOCALAPPDATA\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { reg query "HKCU\SOFTWARE\Google\Chrome\BLBeacon" /v version } if (-not $? -or $? -match $error) { reg query "HKLM\SOFTWARE\Wow6432Node\Microsoft\Windows\CurrentVersion\Uninstall\Google Chrome" /v version }"
Current google-chrome version is UNKNOWN
Get LATEST chromedriver version for UNKNOWN google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_win32.zip
Driver has been s

In [3]:
page = driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')

table = soup.find(id='myTable')
tab_data = [[cell.text for cell in row.find_all(['td', 'th'])]
                                for row in table.find_all("tr")]

df = pd.DataFrame(tab_data[3:], columns=tab_data[2])

cleaned_teams = {}
for i in range(len(list(df['Team']))):
    x = re.search("^.*(?=(\())", str(df['Team'][i]))
    if x:
        cleaned_teams[df['Team'][i]] = x.group(0).strip()
    else:
        print('invalid search')
        
df['Team'].replace(cleaned_teams, inplace=True)
df.head()

,Rk,Team,Eff,FTAR,FT%,FGAR,FG%,3PAR,3P%,MRAR,...,NPAR,NP%,PPSt,PPSC,SCC%,%3PA,%MRA,%NPA,Prox,AP%
0,1,Gonzaga,120.75,23.88,74.66,84.79,53.58,30.74,39.22,20.45,...,33.61,71.19,11.50,11.46,7.65,36.25,24.11,39.63,1.966,1.000
1,2,Arizona,117.84,29.23,72.49,84.50,50.89,28.98,36.74,20.58,...,34.94,70.48,12.17,16.00,7.31,34.29,24.36,41.35,1.929,0.997
2,3,Kentucky,120.12,25.15,72.61,91.57,50.52,24.57,37.96,35.43,...,31.58,70.76,14.01,17.56,9.32,26.83,38.69,34.48,1.923,0.994
3,4,Baylor,116.66,25.26,70.82,90.16,48.17,32.32,36.84,27.10,...,30.74,68.27,15.72,14.37,5.53,35.85,30.06,34.10,2.017,0.992
4,5,Villanova,117.58,28.48,82.74,85.34,46.95,37.64,36.91,17.96,...,29.74,63.48,9.19,12.48,4.07,44.10,21.04,34.85,2.092,0.989


In [12]:
from selenium.webdriver.support.ui import Select

page = driver.get(url)
script = '''
    var myRatings=document.getElementById("cboRatings");
    myRatings.value = "Records";
    myRatings.onchange();
    
    var event = new Event('change');

    // Dispatch it.
    myRatings.dispatchEvent(event);

'''
driver.execute_script(script)

import time
time.sleep(5)


soup = BeautifulSoup(driver.page_source, 'html.parser')



table = soup.find(id='myTable')
tab_data = [[cell.text for cell in row.find_all(['td', 'th'])]
                                for row in table.find_all("tr")]

df2 = pd.DataFrame(tab_data[3:], columns=tab_data[2])

cleaned_teams = {}
for i in range(len(list(df['Team']))):
    x = re.search("^.*(?=(\())", str(df2['Team'][i]))
    if x:
        cleaned_teams[df2['Team'][i]] = x.group(0).strip()
    else:
        print('invalid search')
        
df2['Team'].replace(cleaned_teams, inplace=True)
df2.head()

,Rk,Team,v1-50,v51-100,v101-150,v151-200,v201-250,v251-300,v301+,HQ1,HQ2,HQ3,HQ4,Home,Away,Neut,AP%
0,1,Gonzaga,8-3,4-0,0-0,1-0,4-0,5-0,4-0,9-3,2-0,2-0,13-0,16-0,5-1,5-2,1.000
1,2,Arizona,3-2,9-1,9-0,0-0,4-0,0-0,3-0,5-2,6-1,10-0,7-0,17-0,9-3,2-0,0.997
2,3,Kentucky,9-5,4-1,3-0,1-0,4-0,2-0,2-0,9-6,4-0,4-0,8-0,18-0,6-5,1-1,0.994
3,4,Baylor,12-5,7-0,1-0,0-0,1-0,1-0,4-0,12-4,7-1,1-0,6-0,15-2,8-3,3-0,0.992
4,5,Villanova,8-4,6-3,2-0,3-0,3-0,1-0,0-0,7-6,7-1,4-0,5-0,12-1,8-5,3-1,0.989


In [36]:

def generate_script(key):
    script = '''
        var myRatings=document.getElementById("cboRatings");
        myRatings.value = "{}";
        myRatings.onchange();

        var event = new Event('change');

        // Dispatch it.
        myRatings.dispatchEvent(event);'''.format(key)
    
    return script

def get_data(driver, option):
    page = driver.get(url)
    script = generate_script(option)
    driver.execute_script(script)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    table = soup.find(id='myTable')
    tab_data = [[cell.text for cell in row.find_all(['td', 'th'])]
                                    for row in table.find_all("tr")]

    df2 = pd.DataFrame(tab_data[3:], columns=tab_data[2])

    cleaned_teams = {}
    for i in range(len(list(df['Team']))):
        x = re.search("^.*(?=(\())", str(df2['Team'][i]))
        if x:
            cleaned_teams[df2['Team'][i]] = x.group(0).strip()
        else:
            print('invalid search')

    df2['Team'].replace(cleaned_teams, inplace=True)
    
    return df2
    
    
df = get_data(driver, 'Records')
df.head()

,Rk,Team,v1-50,v51-100,v101-150,v151-200,v201-250,v251-300,v301+,HQ1,HQ2,HQ3,HQ4,Home,Away,Neut,AP%
0,1,Gonzaga,8-3,4-0,0-0,1-0,4-0,5-0,4-0,9-3,2-0,2-0,13-0,16-0,5-1,5-2,1.000
1,2,Arizona,3-2,9-1,9-0,0-0,4-0,0-0,3-0,5-2,6-1,10-0,7-0,17-0,9-3,2-0,0.997
2,3,Kentucky,9-5,4-1,3-0,1-0,4-0,2-0,2-0,9-6,4-0,4-0,8-0,18-0,6-5,1-1,0.994
3,4,Baylor,12-5,7-0,1-0,0-0,1-0,1-0,4-0,12-4,7-1,1-0,6-0,15-2,8-3,3-0,0.992
4,5,Villanova,8-4,6-3,2-0,3-0,3-0,1-0,0-0,7-6,7-1,4-0,5-0,12-1,8-5,3-1,0.989
